In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv('data.csv')
dataset = dataset.drop_duplicates(subset=['id'])
dataset = dataset.reset_index(drop=True)
splitData = dataset.iloc[:, [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]]
print(splitData)
print(dataset)
# dataset.head()

                            artist  ... time_signature
0                         Lil Nas,  ...              4
1     Shawn Mendes, Camila Cabello  ...              4
2                    Billie Eilish  ...              4
3                           Khalid  ...              4
4        Ed Sheeran, Justin Bieber  ...              4
...                            ...  ...            ...
6868                          Cher  ...              4
6869                    Mark Wills  ...              4
6870                   Clay Walker  ...              4
6871               Collective Soul  ...              4
6872                 The Offspring  ...              4

[6873 rows x 16 columns]
                            artist  ... time_signature
0                         Lil Nas,  ...              4
1     Shawn Mendes, Camila Cabello  ...              4
2                    Billie Eilish  ...              4
3                           Khalid  ...              4
4        Ed Sheeran, Justin Bieber  ...

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

splitData.loc[:, 'artist'] = le.fit_transform(splitData['artist'])
splitData.loc[:, 'title'] = le.fit_transform(splitData['title'])
splitData.loc[:, 'genre'] = le.fit_transform(splitData['genre'])
# splitData.head()

#ct = ColumnTransformer(transformers=[('encoder0', OneHotEncoder(), [0])], remainder='passthrough')
#test = np.array(ct.fit_transform(splitData))
#print(test)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [4]:
# feature scaling
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
splitData.head()
arrayForm = np.array(splitData)
print(arrayForm)
arrayForm = ss.fit_transform(arrayForm)
print(arrayForm)
print(splitData)


[[9.76000e+02 3.82500e+03 5.25000e+02 ... 1.36041e+02 1.57067e+05
  4.00000e+00]
 [1.46200e+03 4.46800e+03 1.40300e+03 ... 1.16947e+02 1.90960e+05
  4.00000e+00]
 [1.82000e+02 4.33000e+02 1.09200e+03 ... 1.35128e+02 1.94088e+05
  4.00000e+00]
 ...
 [3.59000e+02 4.55500e+03 1.40300e+03 ... 1.42741e+02 2.01867e+05
  4.00000e+00]
 [3.70000e+02 2.11300e+03 1.15300e+03 ... 1.06929e+02 1.77213e+05
  4.00000e+00]
 [1.61200e+03 5.93400e+03 1.97700e+03 ... 1.07932e+02 1.72467e+05
  4.00000e+00]]
[[ 0.17125898  0.4163394  -1.15777897 ...  0.46715827 -1.60635908
   0.1070023 ]
 [ 1.13238015  0.77855731  0.2548232  ... -0.17658411 -0.85106198
   0.1070023 ]
 [-1.39896779 -1.49445868 -0.24554044 ...  0.43637704 -0.78135527
   0.1070023 ]
 ...
 [-1.04892983  0.82756658  0.2548232  ...  0.69304461 -0.60800219
   0.1070023 ]
 [-1.02717606 -0.54807284 -0.14739838 ... -0.51433476 -1.15741046
   0.1070023 ]
 [ 1.42902248  1.60439162  1.17832393 ... -0.48051924 -1.26317389
   0.1070023 ]]
      artist  ti

In [5]:
# data is now all encoded and such 
# to get the encoded for each then just get arrayForm[index]
    

from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
import json

cosineDist = {}

def getFiveSuggestionsForId(oneIndex, nparray, dataFrame):
  # numpy array 
  base = nparray[oneIndex]
  rowResults = []
  counter = 0
  for row in nparray:
    tempList = []
    tempList.append(dataFrame.at[counter, 'id'])
    tempList.append(dataFrame.at[oneIndex, 'id'])
    theList = tuple(sorted(tempList))

    if (theList not in cosineDist.keys()): 
      cosineDist[theList] = 1 - spatial.distance.cosine(base, row)

    rowResults.append((dataFrame.at[counter, 'id'], cosineDist[theList]))

    counter = counter + 1
  #result = np.where(nparray = )   
  
  sortedResults = sorted(rowResults, key=lambda item: 1 - item[1])
  #print(dataFrame.at[oneIndex, 'title'], '\t', dataFrame.at[oneIndex, 'artist'])
  #print()
  #print('Suggestions: ')

  dataList = []
  for item in range(1, 6):
    indexValue = dataFrame[dataFrame['id']==sortedResults[item][0]].index.values

    #dataList.append((sortedResults[item][0], sortedResults[item][1], dataFrame.at[indexValue[0], 'title'], dataFrame.at[indexValue[0], 'artist']))
    dataList.append(dataFrame.at[indexValue[0], 'artist'] + "\t" + dataFrame.at[indexValue[0], 'title'])
    # print(dataFrame.at[indexValue[0], 'title'], '\t', dataFrame.at[indexValue[0], 'artist'], '\t', sortedResults[item][1])
  return dataList

def getSuggestionsForIds(ids, dataFrame, nparray):
  data = {}
  for id in ids:
    print(id)
    indexValue = dataFrame[dataFrame['id']==id].index.values
    data[dataFrame.at[indexValue[0], 'artist'] + '\t' + dataFrame.at[indexValue[0], 'title']] = getFiveSuggestionsForId(indexValue[0], nparray, dataFrame)

  with open('frontData.json', 'w') as outfile:
    json.dump(data, outfile)

def convertSuggestionsToJson():
  print('not done')

idList = []
for i in range(len(arrayForm)):
  idList.append(dataset.at[i,'id'])
getSuggestionsForIds(idList, dataset, arrayForm)

Streaming output truncated to the last 5000 lines.
74Xdx4ezYQQlg8wKswq25T
6QicTxeiSaRkPnkGDUvYOs
1jk9c0gZI0wbn5GvAj7PBV
1bzM1cd6oqFozdr4wK6HdR
64GRDrL1efgXclrhVCeuA0
2ipJSuGf1GtxSIt2KvOGxl
1KDPWUuA213r6M2WbWUauE
6HMHgBHdLBQ0QYIaOp2gse
6yxgLlHHp9EMklVUUdyjKh
3KGQPtjS7kjEwMizD1bvC8
2bJvI42r8EF3wxjOuDav4r
5Eg4TsPcqNbIjd8ADMZosg
79reBHqpRoWXaQbjBBY3UC
1JDIArrcepzWDTAWXdGYmP
2N3U8uTYhjX2KwGT0pf5x0
4e5lEqydMatcEio6ef9Dpf
7D8owkEcXNUzBytQY0QMPj
3keUgTGEoZJt0QkzTB6kHg
78TTtXnFQPzwqlbtbwqN0y
6br4Pc0BzU6dTD5fEmks2i
6za48riBTylMJHsf4AJPxK
3oZoXyU0SkDldgS7AcN4y4
5jE48hhRu8E6zBDPRSkEq7
1qwnPVOIJjAFfCc40Etb1D
7FC9Lng8iXU081lbigu8m8
20lNo44eUUCnyNNInzwkv9
0N3W5peJUQtI4eyR6GJT5O
3YimpODfNhxj3rN81v4WvE
2pTnrrRF6creKP8lVDpgqw
5fyIGoaaKelzdyW8ELhYJZ
1CS7Sd1u5tWkstBhpssyjP
0k6DnZMLoEUH8NGD5zh2SE
1owHFKK6bWtYYf5V8bD1JR
1B17jo7XD3lJ9AtrdE9kUd
085lgcfCstjd3AjWDAt2OX
5uqphUYsR45kPYplrYSAQh
7795WJLVKJoAyVoOtCWqXN
16PNN9wK1LbedzcTp7y8y0
19cL3SOKpwnwoKkII7U3Wh
2sE61ZmvYH8wiOx5jygkHH
0oPDrdHQwkjaT0aytZavUO
1nHP9w